## Load Data

### Menghubungkan dengan Gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
Data = pd.read_csv('/content/drive/MyDrive/SKRIPSI/Coding/Coding 4/Data_training1.csv')
Data

,datetime,username,content,Sentimen
0,2022-12-30 23:46:37+00:00,Deurafashion,gamis remaja gamis wanita gamis lebaran gamis ...,1
1,2022-12-30 23:28:10+00:00,rosaclaudia_r,cek pink mall tas wanitatas selempang tas han...,1
2,2022-12-30 23:27:36+00:00,rosaclaudia_r,cek funatic dompet lipat koin wanita import mo...,1
3,2022-12-30 23:27:36+00:00,rosaclaudia_r,fashion dengan baju model celana sobek belum m...,2
4,2022-12-30 23:25:51+00:00,rosaclaudia_r,cek dompet kecil pesta kulit coklat premium im...,1
...,...,...,...,...
995,2022-12-30 00:31:47+00:00,TerasTimur,mending kuliner cekrek upload kuncup di inst...,4
996,2022-12-29 23:31:47+00:00,khoirummawarti,salah mbak nourm yaman kuliner namanya wkwkwk ...,4
997,2022-12-29 20:15:56+00:00,FacktaWaw,tahukah anda kuliner orang sunda jawa barat se...,4
998,2022-12-29 15:33:39+00:00,cukkadama,and finally rumah om yono menutup smua kuliner...,4


## Library

In [ ]:
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")
import time
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
import pickle

In [ ]:
pip install NDETCStemmer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.9/111.9 kB 2.4 MB/s eta 0:00:00


## Pre-Processing

In [ ]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   datetime  1000 non-null   object
 1   username  999 non-null    object
 2   content   1000 non-null   object
 3   Sentimen  1000 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [ ]:
import re

# Buat fungsi untuk melakukan case folding
def casefolding(text):
  text = text.lower()                                 # Ubah jadi lowercase
  text = re.sub(r'https?://\S+|www\.\S+', '', text)   # Menghapus URL
  text = re.sub(r'[-+]?[0-9]+', '', text)             # Menghapus angka
  text = re.sub(r'[^\w\s]', '', text)                 # Menghapus tanda baca
  text = text.strip()
  return text


In [ ]:
raw_tex = Data['content'].iloc[5]
case_folding = casefolding(raw_tex)

print('Raw text\t: ', raw_tex)
print('Case folding\t: ', case_folding)


Raw text	:  altercloth  yerin cardi  atasan wanita lengan panjang kardigan korea premium knit rajut import fashion knitwear murah 

  
cek promo
Case folding	:  altercloth  yerin cardi  atasan wanita lengan panjang kardigan korea premium knit rajut import fashion knitwear murah 

  
cek promo


In [ ]:
# Download corpus singkatan
!wget https://raw.githubusercontent.com/ksnugroho/klasifikasi-spam-sms/master/data/key_norm.csv


--2023-08-10 01:23:34--  https://raw.githubusercontent.com/ksnugroho/klasifikasi-spam-sms/master/data/key_norm.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80969 (79K) [text/plain]
Saving to: ‘key_norm.csv’

key_norm.csv        100%[===================>]  79.07K  --.-KB/s    in 0.002s  

2023-08-10 01:23:34 (40.1 MB/s) - ‘key_norm.csv’ saved [80969/80969]



In [ ]:
key_norm = pd.read_csv('key_norm.csv')

# Buat fungsi untuk melakukan word normalization
def text_normalize(text):
  text = ' '.join([key_norm[key_norm['singkat'] == word]['hasil'].values[0] if (key_norm['singkat'] == word).any() else word for word in text.split()])
  text = str.lower(text)
  return text

In [ ]:
raw_tex = Data['content'].iloc[5]
case_folding = casefolding(raw_tex)
word_normalization = text_normalize(case_folding)

print('Raw text\t: ', raw_tex)
print('Case folding\t: ', case_folding)
print('Word Normalization\t: ', word_normalization)

Raw text	:  altercloth  yerin cardi  atasan wanita lengan panjang kardigan korea premium knit rajut import fashion knitwear murah 

  
cek promo
Case folding	:  altercloth  yerin cardi  atasan wanita lengan panjang kardigan korea premium knit rajut import fashion knitwear murah 

  
cek promo
Word Normalization	:  altercloth yerin cardi atasan wanita lengan panjang kardigan korea premium knit rajut import fashion knitwear murah cek promo


In [ ]:
from nltk.corpus import stopwords
stopwords_id = stopwords.words('indonesian')

In [ ]:
len(stopwords_id)

758

In [ ]:
stopwords_id

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [ ]:
# Buat fungsi untuk langkah stopwords removal

def remove_stopwords(text):
  clean_word = []
  all_text = text.split()
  for word in all_text:
    if word not in stopwords_id:
      clean_word.append(word)
  return ' '.join(clean_word)


In [ ]:
raw_tex = Data['content'].iloc[0]
case_folding = casefolding(raw_tex)
word_normalization = text_normalize(case_folding)
stopwords_removal = remove_stopwords(word_normalization)

print('Raw text\t: ', raw_tex)
print('Case folding\t: ', case_folding)
print('Word normalization\t: ', word_normalization)
print('Stopwords removal\t: ', stopwords_removal)

Raw text	:  gamis remaja gamis wanita gamis lebaran gamis pesta gamis kerja amp setelan rok untuk ke kantor yg terbaru amp kekinian cek koleksi gamis deura amp setelan rok deura fashion yuk



deurafashion setelanrok gamisremaja
Case folding	:  gamis remaja gamis wanita gamis lebaran gamis pesta gamis kerja amp setelan rok untuk ke kantor yg terbaru amp kekinian cek koleksi gamis deura amp setelan rok deura fashion yuk



deurafashion setelanrok gamisremaja
Word normalization	:  gamis remaja gamis wanita gamis lebaran gamis pesta gamis kerja amp setelan rok untuk ke kantor yang terbaru amp kekinian cek koleksi gamis deura amp setelan rok deura fashion yuk deurafashion setelanrok gamisremaja
Stopwords removal	:  gamis remaja gamis wanita gamis lebaran gamis pesta gamis kerja amp setelan rok kantor terbaru amp kekinian cek koleksi gamis deura amp setelan rok deura fashion yuk deurafashion setelanrok gamisremaja


Stemming Nondeterministic Context (NDETC) Stemmer

In [ ]:
#import NDETCStemmer library
from NDETCStemmer import NDETCStemmer

#init stemmer
stemmer=NDETCStemmer()

# stemming process
# stemmer=NDETCS.stem()

# Buat fungsi untuk langkah stemming Bahasa Indonesia
def stemming(text):
  text = stemmer.stem(text)
  return text

Model missing, downloading new model....



Downloading...
From: https://drive.google.com/uc?id=1DJ_u_xKSXmgS_CsM0xlB5rIznnDVfz-w
To: /usr/local/lib/python3.10/dist-packages/NDETCStemmer/Model/w2vec_wiki_id_case
100%|██████████| 26.7M/26.7M [00:00<00:00, 27.7MB/s]


Downloading...
From: https://drive.google.com/uc?id=1DQhPp-D3o0e-x3PfJd2Il3vf7wVgZu4J
To: /usr/local/lib/python3.10/dist-packages/NDETCStemmer/Model/w2vec_wiki_id_case.trainables.syn1neg.npy
100%|██████████| 328M/328M [00:11<00:00, 27.9MB/s]


Downloading...
From: https://drive.google.com/uc?id=1zCn5YINEC82cZ1SH-nB4WXUvuELCzKu-
To: /usr/local/lib/python3.10/dist-packages/NDETCStemmer/Model/w2vec_wiki_id_case.wv.vectors.npy
100%|██████████| 328M/328M [00:04<00:00, 66.5MB/s]



Download Complete


In [ ]:
raw_text = Data['content'].iloc[641]
case_folding = casefolding(raw_text)
word_normalization = text_normalize(case_folding)
stopwords_removal = remove_stopwords(word_normalization)
text_stemming = stemming(stopwords_removal)

print('Raw text\t: ', raw_tex)
print('Case folding\t: ', case_folding)
print('Word normalization\t: ', word_normalization)
print('Stopwords removal\t: ', stopwords_removal)
print('Stemming\t: ', text_stemming)

Raw text	:  gamis remaja gamis wanita gamis lebaran gamis pesta gamis kerja amp setelan rok untuk ke kantor yg terbaru amp kekinian cek koleksi gamis deura amp setelan rok deura fashion yuk



deurafashion setelanrok gamisremaja
Case folding	:  produkproduk khas indonesia seperti kain kerajinan tangan hingga karya alat bantu aksesibilitas

tidak hanya hdi expo  berbagai bantuan aksesibilitas juga turut disalurkan bagi para penyandang disabilitas
Word normalization	:  produkproduk khas indonesia seperti kain kerajinan tangan hingga karya alat bantu aksesibilitas tidak hanya hdi expo berbagai bantuan aksesibilitas juga turut disalurkan bagi para penyandang disabilitas
Stopwords removal	:  produkproduk khas indonesia kain kerajinan tangan karya alat bantu aksesibilitas hdi expo bantuan aksesibilitas disalurkan penyandang disabilitas
Stemming	:  produkproduk khas indonesia kain rajin tangan karya alat bantu aksesibilitas hdi expo bantu aksesibilitas salur sandang disabilitas 


In [ ]:
# Buat fungsi untuk menggabungkan seluruh langkah pada text preprocessing
def text_preprocessing_process(text):
  text = casefolding(text)
  text = text_normalize(text)
  text = remove_stopwords(text)
  text = stemming(text)
  return text


In [ ]:
%%time
Data['clean_teks'] = Data['content'].apply(text_preprocessing_process)

# Perhatikan waktu komputasi ketika melakukan text preprocessing

CPU times: user 25 s, sys: 103 ms, total: 25.1 s
Wall time: 38.9 s


In [ ]:
Data.to_csv('clean_teks_NDETCS.csv', index=False)

In [ ]:
Data.clean_teks.iloc[5]

'altercloth yerin cardi atas wanita lengan kardigan korea premium knit rajut import fashion knitwear murah cek promo '

In [ ]:
Data.head()

,datetime,username,content,Sentimen,clean_teks
0,2022-12-30 23:46:37+00:00,Deurafashion,gamis remaja gamis wanita gamis lebaran gamis ...,1,gamis remaja gamis wanita gamis lebaran gamis ...
1,2022-12-30 23:28:10+00:00,rosaclaudia_r,cek pink mall tas wanitatas selempang tas han...,1,cek pink mall tas wanitatas selempang tas hand...
2,2022-12-30 23:27:36+00:00,rosaclaudia_r,cek funatic dompet lipat koin wanita import mo...,1,cek funatic dompet lipat koin wanita import mo...
3,2022-12-30 23:27:36+00:00,rosaclaudia_r,fashion dengan baju model celana sobek belum m...,2,fashion baju model celana sobek jaman mata
4,2022-12-30 23:25:51+00:00,rosaclaudia_r,cek dompet kecil pesta kulit coklat premium im...,1,cek dompet pesta kulit cokelat premium import ...


In [ ]:
X = Data.clean_teks.values
y = Data.Sentimen.values

In [ ]:
# Membuat empty List
processed_tweets = []

for tweet in range(0, len(X)):
    # Hapus semua special characters
    processed_tweet = re.sub(r'\W', ' ', str(X[tweet]))

    # Hapus semua single characters
    processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)

    # Hapus single characters dari awal
    processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet)

    # Substitusi multiple spaces dengan single space
    processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)

    # Hapus prefixed 'b'
    processed_tweet = re.sub(r'^b\s+', '', processed_tweet)

    # Ubah menjadi Lowercase
    processed_tweet = processed_tweet.lower()

    # Masukkan ke list kosong yang telah dibuat sebelumnya
    processed_tweets.append(processed_tweet)

In [ ]:
# Cek sebelum cleaning data
print(str(X[:5]))
print()

# Cek setelah cleaning data
processed_tweets[:5]

['gamis remaja gamis wanita gamis lebaran gamis pesta gamis kerja amp setel rok kantor baru amp kini cek koleksi gamis deura amp setel rok deura fashion yuk deurafashion setelanrok gamisremaja '
 'cek pink mall tas wanitatas selempang tas handbag import tas fashionbaru mini tas harga rupiah dapat shopee '
 'cek funatic dompet lipat koin wanita import model baru fashion korea fa harga rupiah dapat shopee '
 'fashion baju model celana sobek jaman mata '
 'cek dompet pesta kulit cokelat premium import branded murah fashion wab mare korea abg lipat harga rupiah dapat shopee ']



['gamis remaja gamis wanita gamis lebaran gamis pesta gamis kerja amp setel rok kantor baru amp kini cek koleksi gamis deura amp setel rok deura fashion yuk deurafashion setelanrok gamisremaja ',
 'cek pink mall tas wanitatas selempang tas handbag import tas fashionbaru mini tas harga rupiah dapat shopee ',
 'cek funatic dompet lipat koin wanita import model baru fashion korea fa harga rupiah dapat shopee ',
 'fashion baju model celana sobek jaman mata ',
 'cek dompet pesta kulit cokelat premium import branded murah fashion wab mare korea abg lipat harga rupiah dapat shopee ']

## TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'),ngram_range=(1,3))
X1 = tfidfconverter.fit_transform(processed_tweets).toarray()

## Split Data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.1, random_state=0) #data test 5%, data train =95%

## Model

In [ ]:
from sklearn.svm import SVC
text_classifier_svm = SVC(kernel='linear')
t0_svm = time.time()
text_classifier_svm.fit(X_train, y_train)
t1_svm = time.time()

In [ ]:
from sklearn.naive_bayes import GaussianNB
text_classifier_nb = GaussianNB()
t0_nb = time.time()
text_classifier_nb.fit(X_train, y_train)
t1_nb = time.time()

In [ ]:
predictions_nb = text_classifier_nb.predict(X_test)
t2_nb = time.time()
time_linear_train_nb = t1_nb-t0_nb
time_linear_predict_nb = t2_nb-t1_nb

predictions_svm = text_classifier_svm.predict(X_test)
t2_svm = time.time()
time_linear_train_svm = t1_svm-t0_svm
time_linear_predict_svm = t2_svm-t1_svm


print("NB Training time: %fs; Prediction time: %fs" % (time_linear_train_nb, time_linear_predict_nb))
print("SVM Training time: %fs; Prediction time: %fs" % (time_linear_train_svm, time_linear_predict_svm))

NB Training time: 0.014742s; Prediction time: 0.016890s
SVM Training time: 0.339833s; Prediction time: 0.078909s


## Evaluasi

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, roc_auc_score

print("")
print("Naive Bayes")
print('Accuracy  = ', round(accuracy_score(y_test, predictions_nb)*100,2),'%')

print("")
print("Support Vector Machine")
print('Accuracy  = ', round(accuracy_score(y_test, predictions_svm)*100,2),'%')

print("")
print("Naive Bayes")
print(confusion_matrix(y_test,predictions_nb))
print("")
print("Support Vector Machine")
print(confusion_matrix(y_test,predictions_svm))
print("")


print("")
print("Naive Bayes")
print(classification_report(y_test,predictions_nb))
print("")
print("Support Vector Machine")
print(classification_report(y_test,predictions_svm))


Naive Bayes
Accuracy  =  75.0 %

Support Vector Machine
Accuracy  =  81.0 %

Naive Bayes
[[11  0  1  0  0  0  0  0  0  0]
 [ 0  6  0  1  0  0  0  0  0  0]
 [ 1  0  5  0  0  0  0  0  0  0]
 [ 0  0  0  9  0  1  0  0  0  3]
 [ 0  0  0  2 12  1  0  0  0  1]
 [ 0  0  0  0  0  3  0  0  2  0]
 [ 0  0  0  0  0  1 10  0  1  0]
 [ 0  1  0  1  1  0  1  9  1  0]
 [ 0  0  0  0  0  1  0  4  3  0]
 [ 0  0  0  0  0  0  0  0  0  7]]

Support Vector Machine
[[12  0  0  0  0  0  0  0  0  0]
 [ 0  6  0  0  1  0  0  0  0  0]
 [ 0  0  6  0  0  0  0  0  0  0]
 [ 0  0  0 10  3  0  0  0  0  0]
 [ 0  0  0  3 13  0  0  0  0  0]
 [ 0  0  0  0  0  4  1  0  0  0]
 [ 0  0  0  0  0  2 10  0  0  0]
 [ 0  0  0  0  0  0  0 10  4  0]
 [ 0  0  0  0  0  0  0  5  3  0]
 [ 0  0  0  0  0  0  0  0  0  7]]


Naive Bayes
              precision    recall  f1-score   support

           0       0.92      0.92      0.92        12
           1       0.86      0.86      0.86         7
           2       0.83      0.83      0.83    

## Klasifikasi SVM

In [ ]:
# SVM
# Input text
review = "fashion ini bangus dan menarik"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_svm.predict(review_vector)

print(pred_text)

[1]


In [ ]:
# SVM
# Input text
review = "Baju ini tampak buruk"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_svm.predict(review_vector)

print(pred_text)

[2]


In [ ]:
# SVM
# Input text
review = "kulineran di daerah ini lezat dan beragam"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_svm.predict(review_vector)

print(pred_text)

[3]


In [ ]:
# SVM
# Input text
review = "Beli makanan disini makanannya sudah basi dan ngga enak"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_svm.predict(review_vector)

print(pred_text)

[4]


In [ ]:
# SVM
# Input text
review = "ada jasa porter gratis di stasiun pasar senen selama libur natal dan tahun baru  tempometro"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_svm.predict(review_vector)

print(pred_text)

[5]


In [ ]:
# SVM
# Input text
review = "daripada bisnis jasa yang peluangnya ga jelas mending kerja kantoran aja"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_svm.predict(review_vector)

print(pred_text)

[6]


In [ ]:
# SVM
# Input text
review = "Ayo mulai bisnis investasi pada properti"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_svm.predict(review_vector)

print(pred_text)

[7]


In [ ]:
# SVM
# Input text
review = "beli rumah di pinggiran kota rugi karena properti mahal"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_svm.predict(review_vector)

print(pred_text)

[8]


In [ ]:
# SVM
# Input text
review = "kalau kamu kreatif silahkan coba buka bisnis kerajinan tangan"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_svm.predict(review_vector)

print(pred_text)

[9]


In [ ]:
# SVM
# Input text
review = "kerajinan tangan kurang laku ketika harganya mahal"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_svm.predict(review_vector)

print(pred_text)

[0]


## Klasifikasi Naive Bayes

In [ ]:
# Naive Bayes
# Input text
review = "fashion ini bangus dan menarik"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_nb.predict(review_vector)

print(pred_text)

[2]


In [ ]:
# Naive Bayes
# Input text
review = "fashion style baju ini tampak buruk ga sesuai seleraku"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_nb.predict(review_vector)

print(pred_text)

[2]


In [ ]:
# Naive Bayes
# Input text
review = "nasi goreng pak dji kuliner porsi jumbo dengan rasa maknyus di dekat stasiun malang"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_nb.predict(review_vector)

print(pred_text)

[3]


In [ ]:
# Naive Bayes
# Input text
review = "Beli makanan disini makanannya sudah basi dan ngga enak"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_nb.predict(review_vector)

print(pred_text)

[4]


In [ ]:
# Naive Bayes
# Input text
review = "ada jasa porter gratis di stasiun pasar senen selama libur natal dan tahun baru  tempometro"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_nb.predict(review_vector)

print(pred_text)

[5]


In [ ]:
# Naive Bayes
# Input text
review = "Jasa cuci baju disana kurang bagus dan tidak memuaskan hasilnya"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_nb.predict(review_vector)

print(pred_text)

[6]


In [ ]:
# Naive Bayes
# Input text
review = "tiga sektor properti komersial akan terdampak positif pasca pencabutan ppkm oleh presiden jokowi"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_nb.predict(review_vector)

print(pred_text)

[7]


In [ ]:
# Naive Bayes
# Input text
review = "beli rumah di pinggiran kota rugi karena properti mahal"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_nb.predict(review_vector)

print(pred_text)

[8]


In [ ]:
# Naive Bayes
# Input text
review = "sentuhan etnis unik modern berbagai macam model tas anyaman serat alam buatan tangan kerajinan khas indonesia handmade"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_nb.predict(review_vector)

print(pred_text)

[9]


In [ ]:
# Naive Bayes
# Input text
review = "kerajinan tangan kurang laku ketika harganya mahal"

review_vector = tfidfconverter.transform([review]).toarray() # vectorizing
pred_text = text_classifier_nb.predict(review_vector)

print(pred_text)

[0]
